# Import Packages

In [1]:
import numpy as np
import pandas as pd
import glob
import os

# Import CSVs

In [2]:
# Current Path
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '/financial_data'
print(path)

csvfiles = glob.glob(path + "/*.csv")

# Creating a dataframe for each CSV file
dfs = [pd.read_csv(file) for file in csvfiles]

C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\preprocessingHistoricalData/financial_data


# New Dataframe

In [3]:
processed_df_coll = {}

# Pre-Process

In [4]:
prev_row = None
# Up to 0.2% deviation from close price from day before is considered Neutral movement
neutral_percentage = 0.002

for df in dfs:
    processed_df = pd.DataFrame(columns = ['Date', 'OpenPrice', 'ClosePrice', 'PriceDirection'])

    # Creating additional feature columns for each dataframe
    iterables = df.itertuples(index=True, name='Pandas')
    n = df.columns.get_loc('Name') # Name column in df
    df_name = df.iat[1, n] # Name of equity/etf

    print(df_name)

    # Iterating all rows in dataframe
    # Ending at end day - 2 to allow for day + 2 classification label
    for i in range(1, len(df) - 2 ):
        open = df.columns.get_loc('Open')
        close = df.columns.get_loc('Close')
        volume = df.columns.get_loc('Volume')
        df_date = df.columns.get_loc('Date')

        # Prices per row
        open_price = df.iat[i-1, open]
        close_price = df.iat[i-1, close]

        # Price Trend Handling
        price_today = df.iat[i + 0, close]
        price_tomorrow = df.iat[i + 1, close]
        price_2days_after = df.iat[i + 2, close]

        close_price_3d = (price_today + price_tomorrow + price_2days_after)/3
        prev_close_price = df.iat[i - 1, close]
        price_change = close_price_3d - prev_close_price

        if price_change > (close_price_3d * neutral_percentage):
            direction = 'Positive'
        elif price_change < 0 and abs(price_change) > (close_price_3d * neutral_percentage):
            direction = 'Negative'
        else:
            direction = 'Neutral'

        date_ts = df.iat[i,df_date]

        new_row = {'Date':date_ts, 'OpenPrice':open_price, 'ClosePrice':close_price,
                   'PriceDirection':direction}

        #append row to the dataframe
        processed_df = processed_df.append(new_row, ignore_index=True)

    print(processed_df)
    processed_df_coll[df_name] = processed_df


AAL


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      47.28       46.30        Neutral
1     2017-01-05      46.63       46.70       Negative
2     2017-01-06      46.52       45.89       Positive
3     2017-01-09      45.85       46.21       Positive
4     2017-01-10      46.01       47.08       Positive
...          ...        ...         ...            ...
999   2020-12-22      15.70       16.10       Negative
1000  2020-12-23      16.35       15.48       Positive
1001  2020-12-24      15.52       15.89        Neutral
1002  2020-12-28      16.04       15.66       Positive
1003  2020-12-29      15.96       16.06       Negative

[1004 rows x 4 columns]
AAPL


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      28.95       29.04       Positive
1     2017-01-05      28.96       29.00       Positive
2     2017-01-06      28.98       29.15       Positive
3     2017-01-09      29.20       29.48       Positive
4     2017-01-10      29.49       29.75       Positive
...          ...        ...         ...            ...
999   2020-12-22     125.02      128.23       Positive
1000  2020-12-23     131.61      131.88       Positive
1001  2020-12-24     132.16      130.96       Positive
1002  2020-12-28     131.32      131.97       Positive
1003  2020-12-29     133.99      136.69       Negative

[1004 rows x 4 columns]
AMT


            Date  OpenPrice  ClosePrice PriceDirection
0     04/01/2017     106.73      106.15       Negative
1     05/01/2017     106.41      106.34       Negative
2     06/01/2017     106.00      105.97       Negative
3     09/01/2017     105.93      105.27       Negative
4     10/01/2017     105.50      105.02       Negative
...          ...        ...         ...            ...
999   22/12/2020     219.13      219.78       Negative
1000  23/12/2020     219.07      220.49       Negative
1001  24/12/2020     221.50      217.40       Positive
1002  28/12/2020     216.44      218.65       Positive
1003  29/12/2020     218.87      221.07       Positive

[1004 rows x 4 columns]
AVY


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      70.76       70.14       Positive
1     2017-01-05      70.45       72.14        Neutral
2     2017-01-06      72.00       71.43       Positive
3     2017-01-09      72.28       72.40       Positive
4     2017-01-10      72.48       72.56        Neutral
...          ...        ...         ...            ...
999   2020-12-22     150.69      152.08       Positive
1000  2020-12-23     151.37      151.76       Positive
1001  2020-12-24     152.67      152.71       Positive
1002  2020-12-28     153.57      153.40        Neutral
1003  2020-12-29     155.00      153.94        Neutral

[1004 rows x 4 columns]
AWK


...

            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      91.79       91.23        Neutral
1     2017-01-05      90.53       90.42       Positive
2     2017-01-06      90.69       91.13        Neutral
3     2017-01-09      91.39       91.84       Negative
4     2017-01-10      91.09       91.31       Negative
...          ...        ...         ...            ...
999   2020-12-22      84.53       84.17       Negative
1000  2020-12-23      82.53       82.19       Positive
1001  2020-12-24      82.45       82.20        Neutral
1002  2020-12-28      82.16       82.51       Negative
1003  2020-12-29      82.81       82.49       Negative

[1004 rows x 4 columns]
S&P500


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04    2251.57     2257.83       Positive
1     2017-01-05    2261.60     2270.75        Neutral
2     2017-01-06    2268.18     2269.00        Neutral
3     2017-01-09    2271.14     2276.98       Negative
4     2017-01-10    2273.59     2268.90        Neutral
...          ...        ...         ...            ...
999   2020-12-22    3684.28     3694.92        Neutral
1000  2020-12-23    3698.08     3687.26       Positive
1001  2020-12-24    3693.42     3690.01       Positive
1002  2020-12-28    3694.03     3703.06       Positive
1003  2020-12-29    3723.03     3735.36        Neutral

[1004 rows x 4 columns]
WFC


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      55.67       56.00       Negative
1     2017-01-05      56.16       56.05       Negative
2     2017-01-06      55.49       55.18       Negative
3     2017-01-09      55.26       55.04       Negative
4     2017-01-10      54.68       54.24       Positive
...          ...        ...         ...            ...
999   2020-12-22      29.50       29.55       Positive
1000  2020-12-23      29.65       28.96       Positive
1001  2020-12-24      29.29       30.35       Negative
1002  2020-12-28      30.29       29.84        Neutral
1003  2020-12-29      29.94       29.93        Neutral

[1004 rows x 4 columns]
WMT


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      69.24       68.66       Positive
1     2017-01-05      68.66       69.06       Negative
2     2017-01-06      68.43       69.21       Negative
3     2017-01-09      68.41       68.26       Positive
4     2017-01-10      68.33       68.71       Negative
...          ...        ...         ...            ...
999   2020-12-22     145.62      145.97       Negative
1000  2020-12-23     145.18      144.20        Neutral
1001  2020-12-24     143.72      143.22       Positive
1002  2020-12-28     143.54      143.50       Positive
1003  2020-12-29     143.80      145.22       Negative

[1004 rows x 4 columns]
XOM


            Date  OpenPrice  ClosePrice PriceDirection
0     2017-01-04      90.94       90.89       Negative
1     2017-01-05      91.12       89.89       Negative
2     2017-01-06      90.19       88.55       Negative
3     2017-01-09      88.79       88.50       Negative
4     2017-01-10      88.22       87.04       Negative
...          ...        ...         ...            ...
999   2020-12-22      40.99       41.95       Negative
1000  2020-12-23      41.63       41.24       Positive
1001  2020-12-24      41.55       41.77       Negative
1002  2020-12-28      41.65       41.60        Neutral
1003  2020-12-29      41.69       41.74       Negative

[1004 rows x 4 columns]


# Data Export

In [5]:
for key, df in processed_df_coll.items():
    df.to_csv(ROOT_DIR+'\\LSTM\\'+key+".csv", index=False)

print("Export Complete!")

Export Complete!
